In [ ]:
import numpy as np 
import cv2 
import matplotlib.pyplot as plt
import PIL.Image as Image
import gym
import random
import math
from gym import Env, spaces
import time
font = cv2.FONT_HERSHEY_COMPLEX_SMALL 

In [ ]:
model_path = r"/Users/tharhtetsan/Desktop/dynamic-22_4_2022.h5"
model_path = r"/Users/tharhtetsan/Desktop/dynamic-28_4_2022.h5"

In [ ]:
#!pip install gym

In [ ]:
class Point(object):
    def __init__(self, name, x_max, x_min, y_max, y_min):
        self.x = 0
        self.y = 0
        self.x_min = x_min
        self.x_max = x_max
        self.y_min = y_min
        self.y_max = y_max
        self.name = name
    
    def set_position(self, x, y):
        self.x = self.clamp(x, self.x_min, self.x_max - self.icon_w)
        self.y = self.clamp(y, self.y_min, self.y_max - self.icon_h)
    
    def get_position(self):
        return (self.x, self.y)
    
    def move(self, del_x, del_y):
        self.x += del_x
        self.y += del_y
        
        self.x = self.clamp(self.x, self.x_min, self.x_max - self.icon_w)
        self.y = self.clamp(self.y, self.y_min, self.y_max - self.icon_h)

    def clamp(self, n, minn, maxn):
        return max(min(maxn, n), minn)

In [ ]:
class Robot(Point):
    def __init__(self, name, x_max, x_min, y_max, y_min):
        super(Robot, self).__init__(name, x_max, x_min, y_max, y_min)
        self.icon = cv2.imread("robot.PNG") / 255.0
        self.icon_w = 20
        self.icon_h = 20
        self.icon = cv2.resize(self.icon, (self.icon_h, self.icon_w))

In [ ]:
class Obstical(Point):
    def __init__(self, name, x_max, x_min, y_max, y_min):
        super(Obstical, self).__init__(name, x_max, x_min, y_max, y_min)
        self.icon = cv2.imread("building.png") / 255.0
        self.icon_w = 20
        self.icon_h = 20
        self.icon = cv2.resize(self.icon, (self.icon_h, self.icon_w))
        

In [ ]:
class Flag(Point):
    def __init__(self, name, x_max, x_min, y_max, y_min):
        super(Flag, self).__init__(name, x_max, x_min, y_max, y_min)
        self.icon = cv2.imread("flag.PNG") / 255.0
        self.icon_w = 32
        self.icon_h = 32
        self.icon = cv2.resize(self.icon, (self.icon_h, self.icon_w))

In [ ]:
class Robot_Env(Env):
    def __init__(self):
        super(Robot_Env, self).__init__()
        
        # Define a 2-D observation space
        self.observation_shape = (400, 400, 3)
        self.observation_space = spaces.Box(low = np.zeros(self.observation_shape), 
                                            high = np.ones(self.observation_shape),
                                            dtype = np.float16)
        
        # Define an action space ranging from 0 to 4
        self.action_space = spaces.Discrete(6,)
        
         # Create a canvas to render the environment images upon 
        self.canvas = np.ones(self.observation_shape) * 1
        
        self.elements = []
        self.max_fuel = 150
        
        
        self.y_min = int (self.observation_shape[0] * 0.1)
        self.x_min = 0
        self.y_max = int (self.observation_shape[0] * 0.9)
        self.x_max = self.observation_shape[1]
       
    
    
    def step(self, action):
        done = False
        assert self.action_space.contains(action), "Invalid Action"
        # Decrease the fuel counter 
        self.fuel_left -= 1 
        # Reward for executing a step.
       
        
        
        if action == 0:
            self.robot.move(0,5)
        elif action == 1:
            self.robot.move(0,-5)
        elif action == 2:
            self.robot.move(5,0)
        elif action == 3:
            self.robot.move(-5,0)
        elif action == 4:
            self.robot.move(0,0)
            
        
        
        
        
        reward = 0 
        
        for elem in self.elements:
            ## check robot hit building
            if isinstance(elem, Obstical):
                if self.has_collided(self.robot, elem):
                    # Conclude the episode and remove the chopper from the Env.
                    done = True
                    reward = -10
                    #self.elements.remove(self.robot)

            elif isinstance(elem, Flag):
                if self.has_collided(self.robot, elem):
                    done = True
                    reward = 100
                    self.elements.remove(self.robot)

                else:
                    robot_x, robot_y = self.robot.get_position()
                    goal_x,goal_y = elem.get_position()
                    dist =  math.sqrt((goal_x -robot_x)**2+(goal_y-robot_y)**2)
                    reward = (1/dist) * 1000
                        
                        

                
    
         # Increment the episodic return
        self.ep_return += reward
        
        # Draw elements on the canvas
        self.draw_elements_on_canvas()
        
        # If out of fuel, end the episode.
        if self.fuel_left == 0:
            done = True

        return self.canvas, reward, done, []
            
    
    
        
    def reset(self):
        # Reset the fuel consumed
        self.fuel_left = self.max_fuel

        # Reset the reward
        self.ep_return  = 0
        
        
        # Number of building
        self.obstical_count = 0
        
        
        x = random.randrange(int(self.observation_shape[0] * 0.05), int(self.observation_shape[0] * 0.10))
        y = random.randrange(int(self.observation_shape[1] * 0.15), int(self.observation_shape[1] * 0.20))
        
        self.robot = Robot("Robot", self.x_max, self.x_min, self.y_max, self.y_min)
        self.robot.set_position(x,y)
        
        
         # Intialise the elements 
        self.elements = [self.robot]
        
        
        # Reset the Canvas 
        self.canvas = np.ones(self.observation_shape) * 1

        # Draw elements on the canvas
        self.draw_elements_on_canvas()


        # return the observation
        return self.canvas 
    
    
    
    def set_obsticals(self,points):
        for obstical_x, obstical_y in points:
            obstical_obj  =  Obstical("obstical_{}".format(self.obstical_count), self.x_max, self.x_min, self.y_max, self.y_min) 
            obstical_obj.set_position(obstical_x, obstical_y)
            self.elements.append(obstical_obj)
            self.draw_elements_on_canvas()
        return self.canvas

    def set_goal(self,points):
        for flag_x, flag_y in  points:
            flag_obj = Flag("flag_{}".format(self.obstical_count), self.x_max, self.x_min, self.y_max, self.y_min) 
            flag_obj.set_position(flag_x, flag_y)
            self.elements.append(flag_obj)
            self.draw_elements_on_canvas()
        return self.canvas
    
    def draw_elements_on_canvas(self):
        # Init the canvas 
        self.canvas = np.ones(self.observation_shape) * 1

        # Draw the heliopter on canvas
        for elem in self.elements:
            elem_shape = elem.icon.shape
            x,y = elem.x, elem.y
            self.canvas[y : y + elem_shape[1], x:x + elem_shape[0]] = elem.icon

        text = 'Fuel Left: {} | Rewards: {}'.format(self.fuel_left, self.ep_return)

        # Put the info on canvas 
        self.canvas = cv2.putText(self.canvas, text, (10,20), font,  
                   0.8, (0,0,0), 1, cv2.LINE_AA)
    
    
    def get_action_meanings(self):
        return {0: "Right", 1: "Left", 2: "Down", 3: "Up", 4: "Do Nothing"}
    
    def render(self, mode = "human"):
        assert mode in ["human", "rgb_array"], "Invalid mode, must be either \"human\" or \"rgb_array\""
        if mode == "human":
            cv2.imshow("Obstacle Avoidance Robot", self.canvas)
            cv2.waitKey(10)

        elif mode == "rgb_array":
            return self.canvas
        
    def close(self):
        cv2.destroyAllWindows()
        
    def has_collided(self, elem1, elem2):
        x_col = False
        y_col = False

        elem1_x, elem1_y = elem1.get_position()
        elem2_x, elem2_y = elem2.get_position()

        if 2 * abs(elem1_x - elem2_x) <= (elem1.icon_w + elem2.icon_w):
            x_col =  True

        if 2 * abs(elem1_y - elem2_y) <= (elem1.icon_h + elem2.icon_h):
            y_col = True

        if x_col and y_col:
            return True

        return False

In [ ]:
full_obstacle_list = [ (200,100), (400,150),(400, 110), 
                 (430, 390), (201, 304), (135, 281), 
                 (286, 373), (175, 280), (250, 375), (139, 327), (369, 278), 
                 (295, 196), (210, 111),(110,110)]

env = Robot_Env()
obs = env.reset()
obstical_env = env.set_obsticals(full_obstacle_list)
plt.figure(figsize=(10, 10))
plt.imshow(obstical_env)

In [ ]:
from IPython import display

env = Robot_Env()
obs = env.reset()

goal_pos = [[380,380]]
full_obstacle_list = [ (200,100), (400,150),(400, 110), 
                 (430, 390), (201, 304), (135, 281), 
                 (286, 373), (175, 280), (250, 375), (139, 327), (369, 278), 
                 (295, 196), (210, 111),(110,110)]


env.set_obsticals(full_obstacle_list)
env.set_goal(goal_pos)


while True:
    # Take a random action
    action = env.action_space.sample()
    obs, reward, done, info = env.step(action)
    
    # Render the game
    env.render()
    
    if done == True:
        break

env.close()

## Load tf model

In [ ]:
import tensorflow as tf

In [ ]:
def load_my_model(model_path):
    model = tf.keras.models.load_model(model_path)
    return model

In [ ]:
model =  load_my_model(model_path)
model.summary()

### Helper functions

In [ ]:
n_actions = 5
from IPython import display as ipythondisplay

In [ ]:
def pre_process(image_frame):
    
   
    I = image_frame[20:] # Crop
    
    #print("Croped Image size : ",I.shape)
    I = cv2.resize(I,(128,128))
    return I.astype(np.float32)

In [ ]:
def isNan(val):
     return val != val

In [ ]:
test_img = pre_process(obstical_env)
plt.imshow(test_img)
plt.show()

In [ ]:
def random_object_location(num_obj):
    max_x, max_y = 400,400
    obj_width, obj_height = 20,20
    
    obj_x_dist,obj_y_dist = int(max_x/obj_width),int(max_y/obj_height)
    obj_x_list, obj_y_list = [],[]
    
    xs,ys = obj_x_dist,obj_y_dist
    while xs < max_x:
        obj_x_list.append(xs)
        xs = xs+obj_x_dist
    
    while ys < max_y:
        obj_y_list.append(ys)
        ys = ys+obj_x_dist
    
    random_obj_x =  random.sample(obj_x_list, num_obj) 
    random_obj_y =  random.sample(obj_y_list, num_obj) 
    
    random_obj_list = [(x,y) for x,y in zip(random_obj_x,random_obj_y)]
    return random_obj_list

### rl logic

In [ ]:
def choose_action(model,observation):
    
    observation = np.expand_dims(observation,axis = 0)
    
    logits = model.predict(observation)
    
    prob_weights = tf.nn.softmax(logits).numpy()
    
    isContainNan =  [ isNan(x) for x in prob_weights.flatten()]
    
    if True in   isContainNan:
        return np.random.choice(n_actions,size = 1)[0]
    
    action = np.random.choice(n_actions,size = 1, p = prob_weights.flatten())[0]
    
    return action


In [ ]:
class Memory:
    def __init__(self):
        self.clear()
        
    
    def clear(self):
        self.observations = []
        self.actions = []
        self.rewards = []
        
    def add_to_memory(self,new_observation,new_action,new_reward):
        self.observations.append(new_observation)
        
        self.actions.append(new_action)
        
        self.rewards.append(new_reward)

In [ ]:

test_action = choose_action(model,test_img)
print("Predicted output : ",test_action)

In [ ]:
### test model is predict or not

MAX_ITERS = 10
memory = Memory()

In [ ]:
for i_episode in range(MAX_ITERS):
    
    
    # Restart the environment
    full_obstacle_list = random_object_location(15)
    observation = env.reset()
    env.set_obsticals(full_obstacle_list)
    env.set_goal(goal_pos)

    previous_frame = pre_process(observation)
    
    while True :
        current_frame =  pre_process(observation)
        
        obs_change = current_frame - previous_frame
        plt.imshow(obs_change)
        action = choose_action(model,obs_change)
        env.render()
        next_observation, reward, done, info = env.step(action)
        
        memory.add_to_memory(obs_change,action,reward)
        
        if done :
            total_reward = sum(memory.rewards)
            memory.clear()
            
            print("Total Reward : ",total_reward)
            break
        
        observation = next_observation
        previous_frame = current_frame